# Deep Learning
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marcinsawinski/UEP_KIE_DL_CODE/blob/main/dl05_cnn_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Exercise 5
 - Build your own CNN from scratch and try to achieve the highest possible accuracy on MNIST.
 - Use Pretrained Models from Keras, e.g.
 ```python
 model = tf.keras.applications.ResNet50(weights="imagenet")
 ```
 - Transfer stype - follow https://www.tensorflow.org/tutorials/generative/style_transfer



In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import os, datetime 
import numpy as np

In [2]:
# class names are not included with the dataset, so we define them here
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
# scale the pixel intensities to the [0, 1] range
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.
# check the shape of the training set
X_train.shape

(55000, 28, 28)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

In [ ]:
# define the hyperparameters
learning_rate = 0.01
dropout_rate = 0.2
patience = 10


# define the model with different numbers of hidden layers and 10 output neurons and softmax activation
# experiment with different dropout rates and learning rates
model_base = tf.keras.Sequential(
    [
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(dropout_rate, name="dropout1"),
    tf.keras.layers.Dense(300, activation="relu", name="hidden1"),
    # tf.keras.layers.Dropout(dropout_rate, name="dropout2"),
    tf.keras.layers.Dense(100, activation="relu", name="hidden2"),
    # tf.keras.layers.Dropout(dropout_rate, name="dropout3"),
    tf.keras.layers.Dense(10, activation="softmax", name="output"),
    ]
    )

# define the tensorboard callback
ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join("logs", f"fashion_mnist_lr{learning_rate}_do{dropout_rate}_p{patience}{ts}")
tensorboard_cb = tf.keras.callbacks.TensorBoard(logdir,
                                                profile_batch=(100, 200))
# define the early stopping callback
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=patience,
                                                     restore_best_weights=True)
# make a copy of the model so you can use it multiple times
model = tf.keras.models.clone_model(model_base)

# define the optimizer 
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

# train the model
history = model.fit(X_train, y_train, epochs=40,
                    validation_data=(X_valid, y_valid),
                    callbacks=[early_stopping_cb,tensorboard_cb])
# you can plot history to see the training history when not using tensorboard

In [ ]:
# define the hyperparameters
learning_rate = 0.01
dropout_rate = 0.2
patience = 4


# define the model with different numbers of hidden layers and 10 output neurons and softmax activation
# experiment with different dropout rates and learning rates
model_base = tf.keras.Sequential(
    [
    tf.keras.layers.Conv2D(32, kernel_size=3, padding="same",
                           activation="relu", kernel_initializer="he_normal", input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(64, kernel_size=3, padding="same",
                           activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(128, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(10, activation="softmax")
    ], name='cnn'
    )

# define the tensorboard callback
ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join("logs", f"fashion_mnist__cnn_gpu_lr{learning_rate}_do{dropout_rate}_p{patience}{ts}")
tensorboard_cb = tf.keras.callbacks.TensorBoard(logdir,
                                                profile_batch=(100, 200))
# define the early stopping callback
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=patience,
                                                     restore_best_weights=True)
# make a copy of the model so you can use it multiple times
model = tf.keras.models.clone_model(model_base)

# define the optimizer 
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

# train the model
history = model.fit(X_train, y_train, epochs=40,
                    validation_data=(X_valid, y_valid),
                    callbacks=[early_stopping_cb,tensorboard_cb])
# you can plot history to see the training history when not using tensorboard

In [4]:
tf.config.get_visible_devices(
    device_type=None
)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  # Disable all GPUS
  tf.config.set_visible_devices([], 'GPU')
  visible_devices = tf.config.get_visible_devices()
  for device in visible_devices:
    assert device.device_type != 'GPU'
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

# ResNet50

In [5]:
# define the hyperparameters
learning_rate = 0.01
dropout_rate = 0.2
patience = 4
img_shape = (32, 32, 3)
img_shape = (224, 224, 3)

X_train_grey = np.expand_dims(X_train, axis=-1)
X_train_grey = tf.image.resize(X_train_grey, [224,224])
X_train_grey = tf.convert_to_tensor(X_train_grey)
X_train_rgb = tf.image.grayscale_to_rgb(X_train_grey)

X_valid_grey = np.expand_dims(X_valid, axis=-1)
X_valid_grey = tf.image.resize(X_valid_grey, [224,224])
X_valid_grey = tf.convert_to_tensor(X_valid_grey)
X_valid_rgb = tf.image.grayscale_to_rgb(X_valid_grey)


# define the base model for transfer learning
base_model = tf.keras.applications.ResNet50(weights="imagenet", input_shape= img_shape)
base_model.trainable = False
# define the model with different numbers of hidden layers and 10 output neurons and softmax activation
# experiment with different dropout rates and learning rates
model = tf.keras.Sequential(
    [
    base_model,
    tf.keras.layers.BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    tf.keras.layers.Dense(128, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(10, activation="softmax")
    ], name='cnn'
    )

# define the tensorboard callback
ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join("logs", f"fashion_mnist_ResNet50_lr{learning_rate}_do{dropout_rate}_p{patience}{ts}")
tensorboard_cb = tf.keras.callbacks.TensorBoard(logdir,
                                                profile_batch=(100, 200))
# define the early stopping callback
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=patience,
                                                     restore_best_weights=True)

# define the optimizer 
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

# train the model
history = model.fit(X_train_rgb, y_train, epochs=40, batch_size=2,
                    validation_data=(X_valid_rgb, y_valid),
                    callbacks=[early_stopping_cb,tensorboard_cb])
# you can plot history to see the training history when not using tensorboard

Epoch 1/40


2023-04-04 07:35:19.673916: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-04-04 07:35:19.673931: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-04-04 07:35:19.676750: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.
2023-04-04 07:35:19.771497: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [1]:
model.summary()

NameError: name 'model' is not defined

# EfficientNetB0

In [ ]:
# define the hyperparameters
learning_rate = 0.01
dropout_rate = 0.2
patience = 4
img_shape = (32, 32, 3)

X_train_grey = np.expand_dims(X_train, axis=-1)
X_train_grey = tf.image.resize(X_train_grey, [32,32])
X_train_grey = tf.convert_to_tensor(X_train_grey)
X_train_rgb = tf.image.grayscale_to_rgb(X_train_grey)

X_valid_grey = np.expand_dims(X_valid, axis=-1)
X_valid_grey = tf.image.resize(X_valid_grey, [32,32])
X_valid_grey = tf.convert_to_tensor(X_valid_grey)
X_valid_rgb = tf.image.grayscale_to_rgb(X_valid_grey)

base_model.trainable = False
# define the base model for transfer learning
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top= False, weights= "imagenet", input_shape= img_shape, pooling= 'max')

# define the model with different numbers of hidden layers and 10 output neurons and softmax activation
# experiment with different dropout rates and learning rates
model = tf.keras.Sequential(
    [
    base_model,
    tf.keras.layers.BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    tf.keras.layers.Dense(128, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(10, activation="softmax")
    ], name='cnn'
    )

# define the tensorboard callback
ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join("logs", f"fashion_mnist_EfficientNetB0_lr{learning_rate}_do{dropout_rate}_p{patience}{ts}")
tensorboard_cb = tf.keras.callbacks.TensorBoard(logdir,
                                                profile_batch=(100, 200))
# define the early stopping callback
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=patience,
                                                     restore_best_weights=True)

# define the optimizer 
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

# train the model
history = model.fit(X_train_rgb, y_train, epochs=40,
                    validation_data=(X_valid_rgb, y_valid),
                    callbacks=[early_stopping_cb,tensorboard_cb])
# you can plot history to see the training history when not using tensorboard